# Применение парафрейзера rugpt2large
# на параллельном корпусе парафраз RuAdapt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import pandas as pd
# df = pd.read_csv('/content/drive/MyDrive/zlatoust_sentence_aligned_with_CATS.csv')
# df

,source,cos_sim,target,level
0,Ну что ж.,1.000000,Ну что ж.,b2
1,"— Ах, только не мужем, с простою усмешкой сказ...",0.863004,"— Ах, только не мужем, — с простою усмешкой ск...",b1
2,"Мне скоро шестьдесят, я старик, одинокий, ничт...",0.653318,"Ничего во мне нет хорошего, кроме этой любви к...",b2
3,Да!,0.093739,Обстановочка-то какова!,a2_b1
4,"Я вполне уверен, что вы раскаялись и раскаивае...",0.621839,"Я уверен, что вы раскаялись и будете содейство...",b1
...,...,...,...,...
24227,У нас в феврале масленица,0.960889,У нас в феврале масленица.,b1_c1
24228,— Разводиться мне или нет?,0.962737,— Разводиться мне или нет?,b1
24229,"Парусиновую куртку Петр Алексеевич сбросил, ру...",0.938603,"Парусиновую куртку Пётр Алексеевич сбросил, р...",b1_b2
24230,Ай!,1.000000,Ай!,b2


In [ ]:
import pandas as pd
new_df = pd.read_csv('/content/drive/MyDrive/2000_random_rows.csv')
new_df

,Unnamed: 0,source,cos_sim,target
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё."
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн..."
...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч..."
1996,1225,— спросил он,0.568087,— спросил Сергей
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в..."


In [ ]:
!pip install git+https://github.com/RussianNLP/russian_paraphrasers@master

In [ ]:
from russian_paraphrasers import GPTParaphraser
paraphraser = GPTParaphraser(model_name="gpt2", range_cand=False, make_eval=False)

In [ ]:
def get_paraphrase(text):
  paraphrase = paraphraser.generate(
      text, n=1, temperature=1, 
      top_k=10, top_p=0.9, 
      max_length=min(1024, len(text)+10), repetition_penalty=1.5,
      threshold=0.7
    ) 
  return paraphrase['results'][0]['predictions'][0]

In [ ]:
import logging

In [ ]:
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
%%time
new_df['output'] = new_df['source'].apply(get_paraphrase)

CPU times: user 1h 12min 14s, sys: 9.81 s, total: 1h 12min 24s
Wall time: 1h 12min 19s


In [ ]:
# Конвертация в csv-файл вместе с output
# new_df.to_csv("result_gpt2.csv")

# Оценка результата парафрейзера rugpt2large
# с помощью дистрибутивной модели rubert-base-cased-sentence

In [ ]:
pip install sentence-transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [ ]:
import gc
import multiprocessing as mp
from tqdm import tqdm

In [ ]:
import torch
cos = torch.nn.CosineSimilarity(dim=1)

In [ ]:
new_df

,Unnamed: 0,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,У меня есть настоящая жена и ребенок от нее но...
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...",Кити Щерба в ответ эпиграфом к пьесе Булгакова...
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,Когда ему было предложено принять участие в ко...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Всё случившееся случилось за каких-то нескольк...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...",В Саратове женщина получила двухкомнатную квар...
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...",Внук Хрущева рассказал о поездке на поезде с у...
1996,1225,— спросил он,0.568087,— спросил Сергей,"Путин рассказал о том, что его интересует в лю..."
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,Путин пригрозил ударом в случае нападения на С...
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...",я не помню дословно как именно но смысл был такой


In [ ]:
%%time
DF_FILE = '/content/drive/MyDrive/result_gpt2.csv'
# размер одного обрабатываемого кусочка датафрейма
CHUNKSIZE = 20

def process_frame(new_df):
  # функция принимает на вход датафрейм и возвращает косинусные подобия для каждого ряда
  similarities_2 = []
  for n, row in tqdm(new_df.iterrows()):
    s = row['source']
    o = row['output']
    tokens_s = tokenizer([s], return_tensors="pt", padding=True)
    tokens_o = tokenizer([o], return_tensors="pt", padding=True)
    out_s = model(**tokens_s)
    out_o = model(**tokens_o)
    vec_s = torch.mean(out_s.last_hidden_state, dim=1)
    vec_o = torch.mean(out_o.last_hidden_state, dim=1)
    cos_sim = cos(vec_s, vec_o)
    # item() превращает тензор во float
    similarities_2.append(cos_sim.item())
    # на всякий случай собираем ненужное при помощи garbage collector
    gc.collect()
  return similarities_2

# читаем файл по кусочкам заданного размера
reader = pd.read_csv(DF_FILE, chunksize=CHUNKSIZE)
# количество процессов, которые могут идти одновременно
pool = mp.Pool(4)

funclist_2 = []
for new_df_2 in reader:
  # открываем и обрабатываем каждый кусочек
  new_df.columns = ['Unnamed: 0', 'source', 'cos_sim', 'target', 'output']
  f = pool.apply_async(process_frame, [new_df_2])
  funclist_2.append(f)

result_2 = []
for f in funclist_2:
  # таймаут можно увеличить, если возникает ошибка
  sims = f.get(timeout=500)
  result_2.extend(sims)

20it [00:26,  1.34s/it]
20it [00:27,  1.37s/it]
20it [00:27,  1.37s/it]
20it [00:28,  1.41s/it]
20it [00:24,  1.23s/it]
20it [00:26,  1.34s/it]
20it [00:26,  1.31s/it]
20it [00:27,  1.35s/it]
20it [00:26,  1.34s/it]
20it [00:24,  1.22s/it]
20it [00:26,  1.31s/it]
20it [00:26,  1.32s/it]
20it [00:25,  1.29s/it]
20it [00:26,  1.33s/it]
20it [00:27,  1.38s/it]
20it [00:29,  1.47s/it]
20it [00:27,  1.36s/it]
20it [00:27,  1.36s/it]
20it [00:26,  1.31s/it]
20it [00:26,  1.35s/it]
20it [00:24,  1.24s/it]
20it [00:25,  1.27s/it]
20it [00:27,  1.38s/it]
20it [00:27,  1.39s/it]
20it [00:26,  1.32s/it]
20it [00:25,  1.26s/it]
20it [00:26,  1.34s/it]
20it [00:25,  1.27s/it]
20it [00:26,  1.31s/it]
20it [00:24,  1.24s/it]
20it [00:26,  1.33s/it]
20it [00:26,  1.33s/it]
20it [00:25,  1.27s/it]
20it [00:26,  1.33s/it]
20it [00:25,  1.30s/it]
20it [00:26,  1.30s/it]
20it [00:25,  1.28s/it]
20it [00:25,  1.28s/it]
20it [00:25,  1.26s/it]
20it [00:25,  1.29s/it]
20it [00:26,  1.32s/it]
20it [00:26,  1.

CPU times: user 8.05 s, sys: 1.46 s, total: 9.51 s
Wall time: 10min 58s


In [ ]:
result_2[:2000]

[0.8399026393890381,
 0.5885478258132935,
 0.6562736630439758,
 0.6667466163635254,
 0.6495122909545898,
 0.6870946884155273,
 0.7263445258140564,
 0.429727703332901,
 0.7756335735321045,
 0.8382143378257751,
 0.8360720872879028,
 0.8879526853561401,
 0.6808502674102783,
 0.426993191242218,
 0.8622355461120605,
 0.4833355247974396,
 0.6804734468460083,
 0.5193058252334595,
 0.7869707942008972,
 0.5637125372886658,
 0.776608943939209,
 0.71345055103302,
 0.7195637822151184,
 0.8427231311798096,
 0.6188787817955017,
 0.40026092529296875,
 0.7756907939910889,
 0.9201167821884155,
 0.8601241111755371,
 0.6845176815986633,
 0.6308407783508301,
 0.8346184492111206,
 0.698219895362854,
 0.7622290253639221,
 0.8511759042739868,
 0.8587353825569153,
 0.7329282164573669,
 0.5121126174926758,
 0.635636568069458,
 0.7571494579315186,
 0.7222403287887573,
 0.6694915890693665,
 0.7154368162155151,
 0.4786619544029236,
 0.8230712413787842,
 0.7707068920135498,
 0.42768898606300354,
 0.812912106513977

In [ ]:
# Среднее значение
import numpy as np
np.mean(np.array(result_2))

0.6850729276612401

In [ ]:
# Среднеквадратичное отклонение
np.std(np.array(result_2))

0.1330240418751146

# METEOR
https://huggingface.co/spaces/evaluate-metric/meteor

In [ ]:
pip install evaluate

In [ ]:
from evaluate import load

meteor = load("meteor")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_gpt2.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,0,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,У меня есть настоящая жена и ребенок от нее но...
1,1,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...",Кити Щерба в ответ эпиграфом к пьесе Булгакова...
2,2,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,Когда ему было предложено принять участие в ко...
3,3,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Всё случившееся случилось за каких-то нескольк...
4,4,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...",В Саратове женщина получила двухкомнатную квар...
...,...,...,...,...,...
1995,1995,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...",Внук Хрущева рассказал о поездке на поезде с у...
1996,1996,— спросил он,0.568087,— спросил Сергей,"Путин рассказал о том, что его интересует в лю..."
1997,1997,Ударит первым.,0.382794,Начнёт бой первым.,Путин пригрозил ударом в случае нападения на С...
1998,1998,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...",я не помню дословно как именно но смысл был такой


In [ ]:
predictions = final_df['output'].tolist()

In [ ]:
references = final_df['source'].tolist()

In [ ]:
results = meteor.compute(predictions=predictions, references=references)
results

{'meteor': 0.043361811404037075}

# BLEU
https://huggingface.co/spaces/evaluate-metric/bleu

In [ ]:
pip install evaluate

In [ ]:
from evaluate import load

bleu = load("bleu")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_gpt2.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,У меня есть настоящая жена и ребенок от нее но...
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...",Кити Щерба в ответ эпиграфом к пьесе Булгакова...
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,Когда ему было предложено принять участие в ко...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Всё случившееся случилось за каких-то нескольк...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...",В Саратове женщина получила двухкомнатную квар...
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...",Внук Хрущева рассказал о поездке на поезде с у...
1996,1225,— спросил он,0.568087,— спросил Сергей,"Путин рассказал о том, что его интересует в лю..."
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,Путин пригрозил ударом в случае нападения на С...
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...",я не помню дословно как именно но смысл был такой


In [ ]:
output = final_df['output'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
init = []

for word in source:
  init.append(word.split("''"))

init

In [ ]:
predictions = output
print(predictions)

references = init
print(references)

['У меня есть настоящая жена и ребенок от нее но законная дочка все же я', 'Кити Щерба в ответ эпиграфом к пьесе Булгакова "Бег" назвала себя победительницей.', 'Когда ему было предложено принять участие в конкурсе The Fox N, он спрыгнул со страницы популярного британского издания за 2 минуты до старта событий', 'Всё случившееся случилось за каких-то несколько секунд во время представления московского цирка под названием жизнь', 'В Саратове женщина получила двухкомнатную квартиру после пяти лет обысков в коммуналке .', 'Помнишь того француза - поэта и переводчика с французского, который еще недавно жил на твоей квартире', 'Мне показалось что я заметил у него признаки робости перед женщинами".', 'В селе в Донецкой области прогремел взрыв и автоматная стрельба.', 'Зять российского олигарха оказался фанатом французского теннисиста.', 'До свидания мэм удачи', 'Разве ты это еще не понял', 'Не желаете ли вы взять вот это', 'Настя никогда ни о чём таком даже подумать сначала не могла', 'В офи

In [ ]:
bleu_results = bleu.compute(predictions=predictions, references=references)
print(bleu_results)

{'bleu': 0.0, 'precisions': [0.06131038849778374, 0.0015293596812492455, 0.0, 0.0], 'brevity_penalty': 0.7256093660417055, 'length_ratio': 0.757149303401207, 'translation_length': 26847, 'reference_length': 35458}


# SacreBLEU
https://huggingface.co/spaces/evaluate-metric/sacrebleu

In [ ]:
pip install evaluate

In [ ]:
pip install sacrebleu

In [ ]:
from evaluate import load

sacrebleu = load("sacrebleu")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_gpt2.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,У меня есть настоящая жена и ребенок от нее но...
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...",Кити Щерба в ответ эпиграфом к пьесе Булгакова...
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,Когда ему было предложено принять участие в ко...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Всё случившееся случилось за каких-то нескольк...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...",В Саратове женщина получила двухкомнатную квар...
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...",Внук Хрущева рассказал о поездке на поезде с у...
1996,1225,— спросил он,0.568087,— спросил Сергей,"Путин рассказал о том, что его интересует в лю..."
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,Путин пригрозил ударом в случае нападения на С...
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...",я не помню дословно как именно но смысл был такой


In [ ]:
output = final_df['output'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
init = []

for word in source:
  init.append(word.split("''"))

init

In [ ]:
predictions = output
print(predictions)

references = init
print(references)

['У меня есть настоящая жена и ребенок от нее но законная дочка все же я', 'Кити Щерба в ответ эпиграфом к пьесе Булгакова "Бег" назвала себя победительницей.', 'Когда ему было предложено принять участие в конкурсе The Fox N, он спрыгнул со страницы популярного британского издания за 2 минуты до старта событий', 'Всё случившееся случилось за каких-то несколько секунд во время представления московского цирка под названием жизнь', 'В Саратове женщина получила двухкомнатную квартиру после пяти лет обысков в коммуналке .', 'Помнишь того француза - поэта и переводчика с французского, который еще недавно жил на твоей квартире', 'Мне показалось что я заметил у него признаки робости перед женщинами".', 'В селе в Донецкой области прогремел взрыв и автоматная стрельба.', 'Зять российского олигарха оказался фанатом французского теннисиста.', 'До свидания мэм удачи', 'Разве ты это еще не понял', 'Не желаете ли вы взять вот это', 'Настя никогда ни о чём таком даже подумать сначала не могла', 'В офи

In [ ]:
sacrebleu_results = sacrebleu.compute(predictions=predictions, references=references)
print(sacrebleu_results)
print(round(sacrebleu_results["score"], 1))

{'score': 0.028738408025012336, 'counts': [1646, 38, 0, 0], 'totals': [26847, 24847, 22847, 20849], 'precisions': [6.131038849778374, 0.15293596812492455, 0.0021884711340657416, 0.0011990982780948727], 'bp': 0.7256093660417056, 'sys_len': 26847, 'ref_len': 35458}
0.0


# ROUGE
https://huggingface.co/spaces/evaluate-metric/rouge

In [ ]:
pip install evaluate

In [ ]:
pip install rouge_score

In [ ]:
from evaluate import load

rouge = load("rouge")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_gpt2.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,У меня есть настоящая жена и ребенок от нее но...
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...",Кити Щерба в ответ эпиграфом к пьесе Булгакова...
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,Когда ему было предложено принять участие в ко...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Всё случившееся случилось за каких-то нескольк...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...",В Саратове женщина получила двухкомнатную квар...
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...",Внук Хрущева рассказал о поездке на поезде с у...
1996,1225,— спросил он,0.568087,— спросил Сергей,"Путин рассказал о том, что его интересует в лю..."
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,Путин пригрозил ударом в случае нападения на С...
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...",я не помню дословно как именно но смысл был такой


In [ ]:
output = final_df['output'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
predictions = output
print(predictions)

references = source
print(references)

['У меня есть настоящая жена и ребенок от нее но законная дочка все же я', 'Кити Щерба в ответ эпиграфом к пьесе Булгакова "Бег" назвала себя победительницей.', 'Когда ему было предложено принять участие в конкурсе The Fox N, он спрыгнул со страницы популярного британского издания за 2 минуты до старта событий', 'Всё случившееся случилось за каких-то несколько секунд во время представления московского цирка под названием жизнь', 'В Саратове женщина получила двухкомнатную квартиру после пяти лет обысков в коммуналке .', 'Помнишь того француза - поэта и переводчика с французского, который еще недавно жил на твоей квартире', 'Мне показалось что я заметил у него признаки робости перед женщинами".', 'В селе в Донецкой области прогремел взрыв и автоматная стрельба.', 'Зять российского олигарха оказался фанатом французского теннисиста.', 'До свидания мэм удачи', 'Разве ты это еще не понял', 'Не желаете ли вы взять вот это', 'Настя никогда ни о чём таком даже подумать сначала не могла', 'В офи

In [ ]:
rouge_results = rouge.compute(predictions=predictions, references=references, tokenizer=lambda x: x.split())
rouge_results

{'rouge1': 0.02693470671889219,
 'rouge2': 2.2222222222222223e-05,
 'rougeL': 0.026770909269380715,
 'rougeLsum': 0.0268257779874368}

# BERTScore
https://huggingface.co/spaces/evaluate-metric/bertscore

In [ ]:
pip install evaluate

In [ ]:
pip install bert_score

In [ ]:
from evaluate import load

bertscore = load("bertscore")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_gpt2.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,У меня есть настоящая жена и ребенок от нее но...
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...",Кити Щерба в ответ эпиграфом к пьесе Булгакова...
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,Когда ему было предложено принять участие в ко...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Всё случившееся случилось за каких-то нескольк...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...",В Саратове женщина получила двухкомнатную квар...
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...",Внук Хрущева рассказал о поездке на поезде с у...
1996,1225,— спросил он,0.568087,— спросил Сергей,"Путин рассказал о том, что его интересует в лю..."
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,Путин пригрозил ударом в случае нападения на С...
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...",я не помню дословно как именно но смысл был такой


In [ ]:
output = final_df['output'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
predictions = output
print(predictions)

references = source
print(references)

['У меня есть настоящая жена и ребенок от нее но законная дочка все же я', 'Кити Щерба в ответ эпиграфом к пьесе Булгакова "Бег" назвала себя победительницей.', 'Когда ему было предложено принять участие в конкурсе The Fox N, он спрыгнул со страницы популярного британского издания за 2 минуты до старта событий', 'Всё случившееся случилось за каких-то несколько секунд во время представления московского цирка под названием жизнь', 'В Саратове женщина получила двухкомнатную квартиру после пяти лет обысков в коммуналке .', 'Помнишь того француза - поэта и переводчика с французского, который еще недавно жил на твоей квартире', 'Мне показалось что я заметил у него признаки робости перед женщинами".', 'В селе в Донецкой области прогремел взрыв и автоматная стрельба.', 'Зять российского олигарха оказался фанатом французского теннисиста.', 'До свидания мэм удачи', 'Разве ты это еще не понял', 'Не желаете ли вы взять вот это', 'Настя никогда ни о чём таком даже подумать сначала не могла', 'В офи

In [ ]:
bertscore_results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
bertscore_results

{'precision': [0.8590585589408875,
  0.8516319394111633,
  0.8647395372390747,
  0.8691572546958923,
  0.8652021288871765,
  0.8519598245620728,
  0.8638286590576172,
  0.87807297706604,
  0.8376980423927307,
  0.8741086721420288,
  0.8258692622184753,
  0.8924554586410522,
  0.8756426572799683,
  0.8623841404914856,
  0.7983217239379883,
  0.8888500332832336,
  0.8523148894309998,
  0.7842495441436768,
  0.8924309015274048,
  0.8050969243049622,
  0.8809689283370972,
  0.8543247580528259,
  0.8556370735168457,
  0.8919912576675415,
  0.817699670791626,
  0.840781569480896,
  0.8671078681945801,
  0.8669527173042297,
  0.8766524791717529,
  0.8417385816574097,
  0.8718388676643372,
  0.8931636214256287,
  0.8614395260810852,
  0.7508304119110107,
  0.9118980169296265,
  0.8858835101127625,
  0.8349886536598206,
  0.8134341239929199,
  0.8398619890213013,
  0.8425830006599426,
  0.8547906875610352,
  0.8545107841491699,
  0.8211967349052429,
  0.8546951413154602,
  0.887171745300293,
  

In [ ]:
import numpy as np
np.mean(np.array(bertscore_results['precision']))

0.8537000471800565

In [ ]:
import numpy as np
np.mean(np.array(bertscore_results['recall']))

0.8576960211098195

In [ ]:
import numpy as np
np.mean(np.array(bertscore_results['f1']))

0.8554352894425392

# ParaScore
https://github.com/shadowkiller33/ParaScore

In [ ]:
pip install parascore==1.0.5

In [ ]:
from parascore import ParaScorer

In [ ]:
scorer = ParaScorer(lang="ru", model_type='bert-base-uncased')

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_gpt2.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,output
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,У меня есть настоящая жена и ребенок от нее но...
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...",Кити Щерба в ответ эпиграфом к пьесе Булгакова...
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,Когда ему было предложено принять участие в ко...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.",Всё случившееся случилось за каких-то нескольк...
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...",В Саратове женщина получила двухкомнатную квар...
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...",Внук Хрущева рассказал о поездке на поезде с у...
1996,1225,— спросил он,0.568087,— спросил Сергей,"Путин рассказал о том, что его интересует в лю..."
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,Путин пригрозил ударом в случае нападения на С...
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...",я не помню дословно как именно но смысл был такой


In [ ]:
cands = final_df['output'].tolist()
sources = final_df['source'].tolist()
refs = final_df['target'].tolist()

In [ ]:
parascore_results = scorer.base_score(cands, sources, refs, batch_size=16)
parascore_results

[0.7331621217727661,
 0.6879625487327575,
 0.7105689811706543,
 0.6947316098213195,
 0.7099980878829956,
 0.6950734424591064,
 0.684180634021759,
 0.7000267076492309,
 0.6246954369544983,
 0.7685833740234375,
 0.6317668724060058,
 0.7247396874427795,
 0.68358327627182,
 0.7090514469146728,
 0.6437071132659912,
 0.7368214297294616,
 0.7486177849769592,
 0.6684023904800415,
 0.7323554921150207,
 0.616475658416748,
 0.7103884387016296,
 0.6914840507507324,
 0.7305074501037597,
 0.7330108451843261,
 0.6243022847175598,
 0.6574303078651428,
 0.7320092487335205,
 0.7174933719635009,
 0.7079780864715576,
 0.7245797681808471,
 0.7333063054084777,
 0.7633801865577697,
 0.7090335655212402,
 0.6227202582359314,
 0.7832513618469238,
 0.7336989212036132,
 0.6429611015319824,
 0.6126836705207824,
 0.6512665319442749,
 0.6503834891319275,
 0.6856599617004394,
 0.6759072709083557,
 0.6464079189300537,
 0.677313106060028,
 0.7375789451599121,
 0.6185464429855346,
 0.744026916027069,
 0.7087235021591186

In [ ]:
import numpy as np
np.mean(np.array(parascore_results))

0.6951791024208067

In [ ]:
# Среднеквадратичное отклонение
np.std(np.array(parascore_results))

0.0513688788432335